# CS486 - Artificial Intelligence
## Lesson 19 - Approximate Q-Learning

Q-Learning is an model-free way to estimate q-values in MDPs. Q-Learning is also *off-policy*, meaning that it will generate optimal q-values given enough exploration. Here's the Bellman equation that describes how q-values converge. 

$$ Q_{k+1}(s,a) \leftarrow \sum_{s'}T(s,a,s')\left[R(s,a,s')+\gamma \max_{a'} Q_{k}(s',a')\right] $$

In practice, a learning rate is used and our estimated q-values are updated as new samples are received:

$$ Q(s,a) \leftarrow (1-\alpha)Q(s,a)+\alpha\left[r+\gamma \max_{a'} Q(s',a')\right] $$

Today we'll look at ways to approximate Q-Learning to scale better and converge faster. 

In [ ]:
import helpers
from aima.rl import *
from aima.notebook import psource

## Exploration

The first problem we have is exploration. Q-Learning will produce an optimal policy *given enough exploration*. So how much exploration is enough? Even if we execute every action in every state, outcomes in MDPs are uncertain so we can't be sure that the action is optimal unless we execute it many times. How many times should we re-visit a state? 

### $e$-greedy

One approach is to always be exploring. With a probability $e$, take a random action within a state and with a probability $1-e$, take the optimal action. Since Q-Learning is off-policy, the random actions won't degrade the estimated policy, but they will pile on the **regret**. Even when your agent has learned the optimal poliy, an $e$-greedy approach will continue to take sub-optimal actions. 

### Exploration functions 

The idea that we want to **exploit** the information we have learned is not new. Our discount, $\gamma$, decreases rewards over time. Our learning rate, $\alpha$, decreases the impact of infomation over time. Similarly, our **exploration function** decreases the value of exploration over time. 

Exploration functions take the currebt estimated utility of a state and the visit count and returns the *optimistic* value of that state. Essentially an exploration function forces the thorough exploration of a state before returning the staet's true utility. 

### Challenge

Both $e$-greedy and exploration functions are common in machine learning. Which does AIMA's Q-Learning function use?

In [ ]:
psource(QLearningAgent)

## Approximate Q-Learning

Another problem with Q-Learning is that it doesn't generalize across states. Intuitively, we know that experiences in a given state generalize to similar states. But in our planning algorithms the state representation has been the *entire environment at a given moment*. If Q-Learning PacMan needed to look up the q-value for a given state, he would only find it if he had already been in that *exact* state before. 

Instead of storing a value with respect an environmental snapshot, we extract the environmental **features** that are salient to decision-making in that context. So instead storing values for states, we'll store features and decide how valuable a state is given those features. There are two things we need to store values with respect to features: 

1. A feature function, $f(s,a)$. How *active* is this feature? Are we really close to food? Really close to a ghost?
2. Weights for each feature. This is actually what we're learning. How much does a feature matter? Does being close to food impact reward more or less than being close to a ghost?

Once we have these things, we can reformulate our q-values in terms of features:

$$ Q(s,a) = w_1f_1(s,a)+w_2f_2(s,a)+...+w_nf_n(s,a)$$

### Approximate Q-Updates

Instead of updating values for states, we update weights. It looks exactly the same as a q-value update. We adjust the reward by the difference between the reward we received and the reward we thought we'd receive, multiplied by the learning rate:

$$ Q(s,a) = w_1f_1(s,a)+w_2f_2(s,a)+...+w_nf_n(s,a)$$
$$ w_i \leftarrow w_i + \alpha\left[r+\gamma \max_{a'} Q(s',a')-Q(s,a)\right]f_i(s,a) $$

The biggest difference is the feature function, $f(s,a)$. This makes sure that updates to a feature's weight is proportional to its contribution, or *activation*, during the update. 

## Overfitting

Some calculus will formally prove that the online q-updates are just minimizing the error of a linear regression of the feature weights. So why a linear regression and not a quadratic regression? Or some higher order regression? Sometimes that can be a good idea, but often it leads to **overfitting**. Overfitting means that you've feature function so perfectly fits the samples that it only works for the exact samples you've collected and doesn't prediect future samples well. 

In general, keep it simple. Small feature sets and linear updates reduce overfitting. 

## Policy Search

Q-learning optimizes feature weights that reflect the q-values of states. But that doesn't necessarily equate to the best policy. For action selection, we need the ordering of q-values to be correct, not just the relative magnitudes. If we explored forever, the q-values would be perfect and so would their order, but in practice that doesn't happen. So there is a trade-off between modeling (estimating values/q-values) and predication (ordering actions).

We want to learn policies that maximize rewards, not necessarily the values that predict them. Policy search starts with some q-learning then hill-climbs on feature weights to order actions. Basically, we adjust feature weights and evaluate the new policy. 